# Web Scraping

Poniższy kod pobiera ze strony Gumtree dane odnośnie sprzedaży mieszkań w Warszawie. W projekcie autorka wykorzystała bibliotekę BeautifulSoup. Do rozwiązania problemu z serwerem proxy wykorzystano instrukcję ze strony: https://pl.admininfo.info/no-se-ha-podido-establecer-conexi-n-con-el-servidor-proxy-windows-10

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import urllib
import time

In [2]:
url='https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/warszawa/mieszkanie/v1c9073l3200008a1dwp1?priceType=FIXED'


npo_tags = {}
tag_no = 0

while True:

    #Rozwiązanie problemu z serwerem proxy
    headers = {"User-Agent": "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.2.8) Gecko/20100722 Firefox/3.6.8 GTB7.1 (.NET CLR 3.5.30729)", "Referer": "http://example.com"}
    response = requests.get(url, headers=headers, proxies=urllib.request.getproxies(), timeout=5)
    data=response.text
    soup=BeautifulSoup(data, 'html5lib')        
    tags = soup.find_all('div',{'class':'tileV1'})
    
   
    for tag in tags:
            
        link = tag.find('a', {'class': 'href-link'}).get('href') 
        link='https://www.gumtree.pl' + link
        tag_response = requests.get(link)
        tag_data = tag_response.text
        tag_soup = BeautifulSoup(tag_data, 'html5lib') 
            
        #Rozwiązanie problemu, gdy find zwraca None
        node = tag_soup.find('span',{'class':'pre'})
        if node is not None:
            tag_description = tag_soup.find('span',{'class':'pre'}).text
        else:
            tag_description = node
                
        node1=tag_soup.find('span',{'class':'amount'})
        if node1 is not None:
            cena=tag_soup.find('span',{'class':'amount'}).text
        else:
            cena=node1
            
        #Wybór wszystkich wypełnionych pól odnośnie mieszkania
        name_obj=tag_soup.find_all(string=["Data dodania", "Lokalizacja", "Na sprzedaż przez",
        "Rodzaj nieruchomości", "Liczba pokoi", "Liczba łazienek", "Wielkość (m2)", "Parking"])
        #Wybranie tylko unikatowych wartości
        name_obj=list(set(name_obj))
        
            
        i=0
        while i<len(name_obj):
                

            if 'Data dodania' in name_obj:  
                data_dodania = tag_soup.find('div',{'class':'attribute'}).text[12:22]
                i += 1
                
                
            #Pobranie tylko dzielnicy miasta
            if 'Lokalizacja' in name_obj:  
                lokalizacja=tag_soup.find_all('div',{'class':'attribute'})[i].text[11:].split(',')[0] 
                i += 1
                   

            if 'Na sprzedaż przez' in name_obj: 
                na_sprzedaż_przez=tag_soup.find_all('div',{'class':'attribute'})[i].text[17:]
                i += 1
                
            
            if 'Rodzaj nieruchomości' in name_obj: 
                rodz_nieruchomości=tag_soup.find_all('div',{'class':'attribute'})[i].text[20:] 
                i += 1
                

            if 'Liczba pokoi' in name_obj: 
                l_pok=tag_soup.find_all('div',{'class':'attribute'})[i].text[12:] 
                i += 1
                
                
            if 'Liczba łazienek' in name_obj: 
                l_laz=tag_soup.find_all('div',{'class':'attribute'})[i].text[15:] 
                i += 1
                
                
            if 'Wielkość (m2)' in name_obj: 
                wielk=tag_soup.find_all('div',{'class':'attribute'})[i].text[13:] 
                i += 1
                
                
            if 'Parking' in name_obj:   
                parking=tag_soup.find_all('div',{'class':'attribute'})[i].text[7:]
                i += 1


        #Zapisanie danych i wydrukowanie ich
        tag_no+=1
        npo_tags[tag_no] = [link, cena, lokalizacja, wielk , data_dodania, na_sprzedaż_przez, rodz_nieruchomości, l_pok, l_laz, parking, tag_description]
            
        
    url_tag = soup.find('a',{'class':'arrows icon-right-arrow icon-angle-right-gray'}, href=True)
    if url_tag:
        url= 'https://www.gumtree.pl' + url_tag.get('href')
        print(url)
    else:
        break

        
print("Total tags:", tag_no)


npo_tags_df = pd.DataFrame.from_dict(npo_tags, orient = 'index', columns = ['Link', 'Cena', 'Lokalizacja', 'Wielkość (m^2)', 'Data Dodania', 'Na sprzedaż przez', 'Rodzaj nieruchomości', 'Liczba pokoi', 'Liczba łazienek', 'Parking', 'Opis'])



https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/warszawa/mieszkanie/page-2/v1c9073l3200008a1dwp2?priceType=FIXED
https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/warszawa/mieszkanie/page-3/v1c9073l3200008a1dwp3?priceType=FIXED
https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/warszawa/mieszkanie/page-4/v1c9073l3200008a1dwp4?priceType=FIXED
https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/warszawa/mieszkanie/page-5/v1c9073l3200008a1dwp5?priceType=FIXED
https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/warszawa/mieszkanie/page-6/v1c9073l3200008a1dwp6?priceType=FIXED
https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/warszawa/mieszkanie/page-7/v1c9073l3200008a1dwp7?priceType=FIXED
https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/warszawa/mieszkanie/page-8/v1c9073l3200008a1dwp8?priceType=FIXED
https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/warszawa/mieszkanie/page-9/v1c9073l3200008a1dwp9?priceType=FIXED


In [3]:
npo_tags_df

,Link,Cena,Lokalizacja,Wielkość (m^2),Data Dodania,Na sprzedaż przez,Rodzaj nieruchomości,Liczba pokoi,Liczba łazienek,Parking,Opis
1,https://www.gumtree.pl/a-mieszkania-i-domy-spr...,380 000 zł,Bielany,38,19/10/2020,Właściciel,Mieszkanie,2 pokoje,1 łazienka,Ulica,"\n\nSprzedam dwupokojowe mieszkanie 37,70 m2 w..."
2,https://www.gumtree.pl/a-mieszkania-i-domy-spr...,519 000 zł,Włochy,56,15/10/2020,Właściciel,Mieszkanie,2 pokoje,1 łazienka,Ulica,OPIS SZCZEGÓŁOWY NIERUCHOMOŚCI\n\nPOWIERZCHNIA...
3,https://www.gumtree.pl/a-mieszkania-i-domy-spr...,425 000 zł,Białołęka,52,22/09/2020,Właściciel,Mieszkanie,3 pokoje,1 łazienka,Ulica,BEZPOŚREDNIO OD DEWELOPRA - WYKOŃCZONE POD KLU...
4,https://www.gumtree.pl/a-mieszkania-i-domy-spr...,479 900 zł,Ochota,36,20/10/2020,Agencja,Mieszkanie,Kawalerka lub garsoniera,1 łazienka,Ulica,Opis lokalu:\n1 pokój o powierzchni 36 mkw. po...
5,https://www.gumtree.pl/a-mieszkania-i-domy-spr...,570 000 zł,Wola,47,20/10/2020,Agencja,Mieszkanie,2 pokoje,1 łazienka,Ulica,Sprzedam bezpośrednio (bez prowizji oraz bez p...
...,...,...,...,...,...,...,...,...,...,...,...
1078,https://www.gumtree.pl/a-mieszkania-i-domy-spr...,544 687 zł,Włochy,29,20/10/2020,Agencja,Mieszkanie,Kawalerka lub garsoniera,2 łazienki,Kryty,Parametry lokalu odpowiadają pozostałym lokalo...
1079,https://www.gumtree.pl/a-mieszkania-i-domy-spr...,460 801 zł,Włochy,25,20/10/2020,Agencja,Mieszkanie,Kawalerka lub garsoniera,2 łazienki,Kryty,Parametry lokalu odpowiadają pozostałym lokalo...
1080,https://www.gumtree.pl/a-mieszkania-i-domy-spr...,603 450 zł,Mokotów,40,20/10/2020,Agencja,Mieszkanie,2 pokoje,1 łazienka,Kryty,"Na sprzedaż dwupokojowe, umeblowane mieszkanie..."
1081,https://www.gumtree.pl/a-mieszkania-i-domy-spr...,377 868 zł,Włochy,20,20/10/2020,Agencja,Mieszkanie,Kawalerka lub garsoniera,1 łazienka,Kryty,Parametry lokalu odpowiadają pozostałym lokalo...


In [4]:
#Sprawdzenie czy nie ma jakiś nulli
npo_tags_df[npo_tags_df['Cena'].notnull()==False]

,Link,Cena,Lokalizacja,Wielkość (m^2),Data Dodania,Na sprzedaż przez,Rodzaj nieruchomości,Liczba pokoi,Liczba łazienek,Parking,Opis


In [5]:
#Zapisanie danych do pliku csv
npo_tags_df.to_csv('npo_tags_df.csv', sep=';', index = False, header=True)